# Metadata Replacement Retriever
- Pros:
  - pass
- Cons:
  - pass

In [1]:
from llama_index.core import Settings
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.llms.ollama import Ollama

ollama_embedding = OllamaEmbedding(
    model_name="mxbai-embed-large",
    base_url="http://localhost:11434",
)

ollama = Ollama(
    model="llama3.2:3b-instruct-fp16",
    base_url="http://localhost:11434"
)

Settings.llm = ollama
Settings.embed_model = ollama_embedding

In [2]:
from llama_index.core.node_parser import SentenceWindowNodeParser
from llama_index.core.node_parser import SentenceSplitter

# create the sentence window node parser w/ default settings
node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=3,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
)

# base node parser is a sentence splitter
text_splitter = SentenceSplitter()

Settings.text_splitter = text_splitter

In [3]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["../documents/ods-cpp (1).pdf"]
).load_data()

In [4]:
nodes = node_parser.get_nodes_from_documents(documents)
base_nodes = text_splitter.get_nodes_from_documents(documents)


In [5]:
from llama_index.core import VectorStoreIndex

sentence_index = VectorStoreIndex(nodes)
base_index = VectorStoreIndex(base_nodes)

In [6]:
from llama_index.core.postprocessor import MetadataReplacementPostProcessor

query_engine = sentence_index.as_query_engine(
    similarity_top_k=2,
    # the target key defaults to `window` to match the node_parser's default
    node_postprocessors=[
        MetadataReplacementPostProcessor(target_metadata_key="window")
    ],
)
window_response = query_engine.query(
    "What is the difference between stack and queue?"
)
print(window_response)

A Stack and a Queue are two different data structures that serve distinct purposes. 

The primary difference between them lies in their ordering of elements. In a Stack, elements are added and removed from the top of the structure, whereas in a Queue, elements are added to one end and removed from the other. This results in a LIFO (Last-In-First-Out) ordering for Stacks and an FIFO (First-In-First-Out) ordering for Queues.

Another difference lies in how elements can be added or removed from them. A Stack has only two operations, addFirst(x) and removeFirst(), whereas a Queue also includes remove() with no argument to indicate that the next element should be removed regardless of its position.


In [7]:
window = window_response.source_nodes[0].node.metadata["window"]
sentence = window_response.source_nodes[0].node.metadata["original_text"]

print(f"Window: {window}")
print("------------------")
print(f"Original Sentence: {sentence}")

Window: Interfaces §1.2
x · · ·
add (x)/enqueue (x) remove ()/dequeue ()
Figure 1.1: A FIFO Queue .
 1.2.1 The Queue ,Stack , and Deque Interfaces
TheQueue interface represents a collection of elements to which we can
add elements and remove the next element.  More precisely, the opera-
tions supported by the Queue interface are
•add(x): add the value xto the Queue
•remove (): remove the next (previously added) value, y, from the
Queue and return y
Notice that the remove () operation takes no argument.  The Queue ’squeue-
ing discipline decides which element should be removed.  There are many
possible queueing disciplines, the most common of which include FIFO,
priority, and LIFO.

------------------
Original Sentence: 1.2.1 The Queue ,Stack , and Deque Interfaces
TheQueue interface represents a collection of elements to which we can
add elements and remove the next element. 


In [8]:
query_engine = base_index.as_query_engine(similarity_top_k=2)
vector_response = query_engine.query(
    "What is the difference between stack and queue?"
)
print(vector_response)

The primary difference between a Stack and a Queue lies in the order in which elements are removed. In a Stack, the most recently added element is removed first, whereas in a Queue, the oldest element is typically removed first. This is often referred to as a Last-In-First-Out (LIFO) discipline for stacks and a First-In-First-Out (FIFO) discipline for queues.


In [9]:
query_engine = base_index.as_query_engine(similarity_top_k=5)
vector_response = query_engine.query(
    "What is the difference between stack and queue?"
)
print(vector_response)

Two structures that are often confused with each other due to their similar naming conventions but distinct behaviors.

A Stack follows a Last-In-First-Out (LIFO) discipline, meaning that the most recently added element is also the next one removed. This structure is so common that it gets its own name: Stack. In contrast, a Queue follows a First-In-First-Out (FIFO) discipline, where elements are removed in the same order they were added.

In other words, when you add an element to a stack, it goes on top of the existing elements. When you remove an element from a stack, you take the one at the top. In contrast, when you add an element to a queue, it gets added to the end of the line. When you remove an element from a queue, you take the one that was added first.

These two disciplines lead to different operations and use cases for stacks and queues. Stacks are often used in parsing expressions, evaluating postfix notation, or implementing recursive algorithms iteratively, while queues

In [10]:
for source_node in window_response.source_nodes:
    print(source_node.node.metadata["original_text"])
    print("--------")

1.2.1 The Queue ,Stack , and Deque Interfaces
TheQueue interface represents a collection of elements to which we can
add elements and remove the next element. 
--------
It is worth
noting that a Stack can be implemented using only addFirst (x) and
removeFirst () while a FIFO Queue can be implemented using addLast (x)
andremoveFirst ().

--------


In [11]:
for node in vector_response.source_nodes:
    print("AMOC mentioned?", "AMOC" in node.node.text)
    print("--------")

AMOC mentioned? False
--------
AMOC mentioned? False
--------
AMOC mentioned? False
--------
AMOC mentioned? False
--------
AMOC mentioned? False
--------


In [ ]:
print(vector_response.source_nodes[2].node.text)
